In [ ]:
import pandas as pd
import sqlalchemy
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
# Convertig Excel file into a Dataframe
main_data = pd.read_excel(r'E:\Mini_project_2\FAOSTAT_data.xlsx')

**Droping unwanted columns**

In [4]:
main_data.drop(columns='Flag',inplace=True)

In [5]:
main_data.drop(columns='Flag Description',inplace=True)

In [6]:
main_data.drop(columns='Note',inplace=True)

In [7]:
main_data.drop(columns='Year Code',inplace=True)

In [ ]:
#Checking null values
main_data.isna().sum()

In [ ]:
IC_null = main_data[main_data['Item Code (CPC)'].isna()]

In [ ]:
IC_check= main_data[(main_data['Area']== 'Afghanistan') & (main_data['Item']=='Oranges')]


In [ ]:
#Forward filling for item code column

main_data['Item Code (CPC)'].fillna(method='ffill',inplace=True)

In [ ]:
main_data[main_data['Unit'].isna()]

In [ ]:
Unit_check1=main_data[(main_data['Area']== 'Afghanistan') & (main_data['Element']== 'Production')  & (main_data['Item']=='Hen eggs in shell, fresh') ]

In [ ]:
Unit_check2=main_data[(main_data['Area']== 'Afghanistan') & (main_data['Element']== 'Yield')  & (main_data['Item']=='Linseed') ]

In [ ]:
# Find indexes of NaN values in a "Unit" column
unit_index = main_data[main_data["Unit"].isna()].index

In [16]:
# Replace NaN at specific positions with different values
main_data.loc[unit_index[0], "Unit"] = "t"
main_data.loc[unit_index[1], "Unit"] = "kg/ha"

**Handling values column**

In [17]:
vaule_nan=main_data[main_data['Value'].isna()]

In [19]:
# Droping null value rows
data=main_data.dropna()

In [ ]:
#Checking null values
data.isna().sum()

In [ ]:
#droping rows with '1000 no'
data.drop(data[data['Unit'] =='1000 No'].index, inplace=True)


In [ ]:
# Filtering rows with elemets 'Area harvested', 'Yield','Production'. 
data_fil = data[data['Element'].isin(['Area harvested', 'Yield','Production'])]

In [ ]:
# selecting columns for transform
pre_trasform=data_fil[['Domain Code','Domain','Area Code (M49)','Element','Area','Item Code (CPC)','Item','Year','Value']]

**Transforming dataset**

In [25]:
df_transformed = pre_trasform.pivot_table(
    index=["Domain Code", "Domain", "Area Code (M49)", "Area", "Item Code (CPC)", 
           "Item","Year"],
    columns="Element",
    values="Value",
    aggfunc="first"  
).reset_index()
# Use 'first' to avoid aggregation issues

In [ ]:
#Cheacking null values
df_transformed.isna().sum()

In [27]:
#where no prodction monitored
no_production=df_transformed[df_transformed['Production'].isna()]

In [28]:
#Droping the rows with no production
FOA_Data=df_transformed.dropna(subset=['Production'])

In [30]:
#selecting columns
FOA_Data_complete=FOA_Data[['Area','Item','Year','Area harvested',
                   'Yield','Production']]

In [32]:
#Droping rows with null values
FOA_Data_final=FOA_Data_complete.dropna()

**Final dataset**

In [ ]:
FOA_Data_final.columns

In [ ]:
#checking duplicates
print(FOA_Data_final.duplicated().sum())

In [ ]:
#checking null values
FOA_Data_final.isna().sum()

In [ ]:
FOA_Data_final.info()

**Transferring Data to SQL**

In [ ]:

from sqlalchemy import create_engine
host = "localhost"
port ="5432"
username = "postgres" 
password = "begin25"
db_name = "Crop_data"

connector =f"postgresql://{username}:{password}@{host}:{port}/{db_name}" #must follow this syntax

engine = create_engine(connector)

table_name ="FAO_data"

FOA_Data_final.to_sql(table_name,engine,if_exists="replace",index= False)

print("Data Transferred Successfully")

***EDA***

Analyze Crop Distribution

In [ ]:
#Crop Types
ct=FOA_Data_final['Item'].value_counts(ascending=False).reset_index(name='Cultivation_count')
top10_max=ct.head(10)

In [ ]:
ct_min=FOA_Data_final['Item'].value_counts(ascending=True).reset_index(name='Cultivation_count')
top10_min=ct_min.head(10)

In [ ]:
max_crop=px.bar(top10_max,x='Item',y='Cultivation_count')
max_crop.update_layout(title='Maximum cultivated items during the year 2019 to 2023')
max_crop.show()

In [ ]:
min_crop=px.bar(top10_min,x='Item',y='Cultivation_count')
min_crop.update_layout(title='Minimum cultivated items during the year 2019 to 2023')
min_crop.show()

In [ ]:
#Geographical Distribution

gd=FOA_Data_final[['Area','Item']]

In [ ]:
gd_analysis=gd.value_counts().reset_index(name='cultivated_count')

In [ ]:
 #idxmax() - to find highest cultivated crop type in each country
gd_most = gd_analysis.loc[gd_analysis.groupby("Area")["cultivated_count"].idxmax()] 

In [ ]:
fig_gd = px.scatter_geo(gd_most, 
                     locations="Area", 
                     locationmode="country names", 
                     hover_data={'Item','cultivated_count'},
                     title="Geographical Distribution by Country",
                     projection="orthographic")

fig_gd.show()

**Temporal Analysis**

In [ ]:
#Yearly Trends & Growth Analysis

yt=FOA_Data_final[['Area','Year','Area harvested','Yield','Production']]

In [ ]:
year_Area_harvested = yt.groupby(["Area", "Year"])["Area harvested"].sum().reset_index()

In [ ]:
year_Yield=yt.groupby(["Area", "Year"])["Yield"].sum().reset_index()

In [ ]:
year_Production=yt.groupby(["Area", "Year"])["Production"].sum().reset_index()

In [ ]:
yt_Area_harvested = px.line(year_Area_harvested, x="Year", y="Area harvested", color="Area", markers=True, 
              title="Yearly Trends in Area harvested across the Countries")
yt_Area_harvested.show()

In [ ]:
yt_Yield = px.line(year_Yield, x="Year", y="Yield", color="Area", markers=True, 
              title="Yearly Trends in Yield across the Countries")
yt_Yield.show()


In [ ]:
yt_Production = px.line(year_Production, x="Year", y="Production", color="Area", markers=True, 
              title="Yearly Trends in Production(tons) across the Countries")
yt_Production.show()

**Input-Output Relationships**

In [99]:
correlation_check = FOA_Data_final[['Area harvested','Yield','Production']]

In [ ]:
import seaborn as sns

# Plot heatmap for correlation
plt.figure(figsize=(6,4))
sns.heatmap(correlation_check.corr(), annot=True,vmax=1,vmin=-1)
plt.title("Correlation Heatmap")
plt.show()


**Comparative Analysis**

In [ ]:
#Across Crops

com_analysis_yield=FOA_Data_final[['Item','Yield']]

In [ ]:
total_yield=com_analysis_yield.groupby('Item')['Yield'].sum().reset_index()

In [113]:
high_yield=total_yield.sort_values(by="Yield", ascending=False)
low_yield=total_yield.sort_values(by="Yield", ascending=True)

In [ ]:
fig_hyc=px.bar(high_yield.head(10),x='Item',y='Yield',color="Item", title="Highest yield Crops")
fig_hyc.show()

In [ ]:
fig_lyc=px.bar(low_yield.head(10),x='Item',y='Yield',color="Item", title="Lowest yield Crops")
fig_lyc.show()

In [ ]:
productive_analysis=FOA_Data_final[['Area','Production']]

In [ ]:
total_Production=productive_analysis.groupby('Area')['Production'].sum().reset_index()

In [ ]:
high_production=total_Production.sort_values(by="Production", ascending=False)

In [ ]:
fig=px.bar(high_production.head(10),x='Area',y='Production',color="Area", title="Highly Productive Regions(2019-2023)")
fig.show()

In [ ]:
fig_geo=px.scatter_geo(high_production,locations='Area',locationmode='country names',hover_data='Production',size='Production',color='Area',
                   title="Highly Productive Regions(2019-2023)", projection="orthographic")
fig_geo.show()

**Productivity Analysis**

In [101]:
yield_analysis=FOA_Data_final[['Area','Item','Yield']]

In [ ]:
yield_analysis_total=yield_analysis.groupby(['Area','Item'])['Yield'].sum().reset_index()

In [ ]:
productive_analysis_yield=yield_analysis_total.sort_values(by="Yield", ascending=False)
productive_analysis_yield.head(10)

In [ ]:
fig = px.bar(productive_analysis_yield.head(10), x="Item", y="Yield", color="Area", title="High yield by Crop & Region")
fig.show()


**Outlier Detection**

In [ ]:
#Outlier Area harvested column
Q1_Area_harvested=FOA_Data_final['Area harvested'].quantile(0.25)
Q3_Area_harvested=FOA_Data_final['Area harvested'].quantile(0.75)

IQR_Area_harvested=Q3_Area_harvested-Q1_Area_harvested

print(f'Q1_Area_harvested:{Q1_Area_harvested}\nQ3_Area_harvested:{Q3_Area_harvested}\nIQR_Area_harvested:{IQR_Area_harvested}')

In [ ]:
#Outlier Yield column
Q1_Yield=FOA_Data_final['Yield'].quantile(0.25)
Q3_Yield=FOA_Data_final['Yield'].quantile(0.75)

IQR_Yield=Q3_Yield-Q1_Yield

print(f'Q1_Yield:{Q1_Yield}\nQ3_Yield:{Q3_Yield}\nIQR_Yield:{IQR_Yield}')

In [ ]:
#Outlier Production column
Q1_Production=FOA_Data_final['Production'].quantile(0.25)
Q3_Production=FOA_Data_final['Production'].quantile(0.75)

IQR_Production=Q3_Production-Q1_Production

print(f'Q1_Production:{Q1_Production}\nQ3_Production:{Q3_Production}\nIQR_Production:{IQR_Production}')

In [ ]:
lb_AreaHarvested= Q1_Area_harvested - 1.5 * IQR_Area_harvested

up_AreaHarvested= Q3_Area_harvested + 1.5 * IQR_Area_harvested

print(f'Area harvested Outlier \n lower:{lb_AreaHarvested} \n upper:{up_AreaHarvested}')

In [ ]:
lb_yield= Q1_Yield - 1.5 * IQR_Yield

up_yield= Q3_Yield + 1.5 * IQR_Yield

print(f'Yield Outlier \n lower:{lb_yield} \n upper:{up_yield}')


In [ ]:
lb_Production= Q1_Production - 1.5 * IQR_Production

up_Production= Q3_Production + 1.5 * IQR_Production

print(f'Production Outlier \n lower:{lb_Production} \n upper:{up_Production}')


In [171]:
FOA_outlier_data=FOA_Data_final

In [ ]:
FOA_outlier_data["AreaHarvested_outlier"]=(FOA_outlier_data['Area harvested'] < lb_AreaHarvested) | (FOA_outlier_data['Area harvested'] > up_AreaHarvested)

In [ ]:
# Filtering columns Outlier == True
AreaHarvested_outlier=FOA_outlier_data[FOA_outlier_data['AreaHarvested_outlier']==True]
AreaHarvested_outlier

In [ ]:
FOA_outlier_data["yield_outlier"]=(FOA_outlier_data['Yield'] < lb_yield) | (FOA_outlier_data['Yield'] > up_yield)

In [ ]:
# Filtering columns Outlier == True
yield_outlier=FOA_outlier_data[FOA_outlier_data['yield_outlier']==True]

In [ ]:
FOA_outlier_data["Production_outlier"]=(FOA_outlier_data['Production'] < lb_Production) | (FOA_outlier_data['Production'] > up_Production)

In [ ]:
# Filtering columns Outlier == True
Production_outlier=FOA_outlier_data[FOA_outlier_data['Production_outlier']==True]

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 4))

axes[0].boxplot(FOA_outlier_data['Area harvested'])
axes[0].set_title('Outlier Area harvested')

axes[1].boxplot(FOA_outlier_data['Yield'])
axes[1].set_title('Outlier Yield')

axes[2].boxplot(FOA_outlier_data['Production'])
axes[2].set_title('Outlier Production')

plt.tight_layout()
plt.show()

**Machine Learning(Regression)**

In [ ]:
final_data=FOA_Data_final[['Area','Item','Year','Area harvested','Yield','Production']]

In [ ]:
#Encoding columns with object datatype

from sklearn.preprocessing import LabelEncoder

lb_en_area=LabelEncoder()
lb_en_Item=LabelEncoder()


final_data['Area']=lb_en_area.fit_transform(final_data['Area'])
final_data['Item']=lb_en_Item.fit_transform(final_data['Item'])


In [ ]:
# Splitting data into Features and Target

x=final_data.drop(columns='Production',axis=1) #Features
y=final_data['Production'] #Target

In [ ]:
# Splitting Features and Target for Training,Testing

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

In [ ]:
#LinearRegression model

from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
#DecisionTreeRegressor model

from sklearn.tree import  DecisionTreeRegressor
dtr=DecisionTreeRegressor()
dtr.fit(x_train,y_train)

In [ ]:
#KNeighborsRegressor model

from sklearn.neighbors import KNeighborsRegressor

KNN=KNeighborsRegressor(n_neighbors=5)
KNN.fit(x_train,y_train)

In [ ]:
#Hyperparameter tuning

linear_param={'fit_intercept':[True,False]}

decision_param={"splitter":['best', 'random'],"max_depth":[None,5,10,15,20],"min_samples_split":[2,3,4,5,6,7,8,10]}

knn_param={'n_neighbors':[5,7,9,12,15,18,20],"weights":['uniform','distance']}


In [ ]:
#Creating multiple models

from sklearn.model_selection import GridSearchCV

grid_search_lr=GridSearchCV(lr,linear_param,cv=5)
grid_search_dtr=GridSearchCV(dtr,decision_param,cv=5)
grid_search_KNN=GridSearchCV(KNN,knn_param,cv=5)

In [ ]:
#Training multiple models

grid_search_lr.fit(x_train,y_train)
grid_search_dtr.fit(x_train,y_train)
grid_search_KNN.fit(x_train,y_train)

In [ ]:
#To find Best parameters for KNeighborsRegressor model
grid_search_KNN.best_estimator_

In [ ]:
KNN_grid=KNeighborsRegressor(n_neighbors=7, weights='distance')
KNN_grid.fit(x_train,y_train)

In [ ]:
#To find Best parameters for DecisionTreeRegressor model
grid_search_dtr.best_estimator_

In [ ]:
grid_dtr=DecisionTreeRegressor(max_depth=15,  min_samples_split=3, splitter='random')
grid_dtr.fit(x_train,y_train)

In [ ]:
#DecisionTreeRegressor model 
grid_dtr1=DecisionTreeRegressor()
grid_dtr1.fit(x_train,y_train)

In [ ]:
#RandomForestRegressor model
from sklearn.ensemble import RandomForestRegressor
rfc=RandomForestRegressor()
rfc.fit(x_train,y_train)

In [ ]:
# Error metrics and Scores of the different models

from sklearn.metrics import mean_absolute_error

y_pred_lr=lr.predict(x_test)
lr_score=lr.score(x_test,y_test)
linear_mae=mean_absolute_error(y_test,y_pred_lr)

y_pred_dtr=dtr.predict(x_test)
dtr_score=dtr.score(x_test,y_test)
dtr_mae=mean_absolute_error(y_test,y_pred_dtr)

y_pred_knn=KNN.predict(x_test)
KNN_score=KNN.score(x_test,y_test)
knn_mae=mean_absolute_error(y_test,y_pred_knn)


y_pred_KNN_grid=KNN_grid.predict(x_test)
KNN_grid_score=KNN_grid.score(x_test,y_test)
KNN_grid_mae=mean_absolute_error(y_test,y_pred_KNN_grid)


y_pred_grid_dtr=grid_dtr.predict(x_test)
grid_dtr_score=grid_dtr.score(x_test,y_test)
grid_dtr_mae=mean_absolute_error(y_test,y_pred_grid_dtr)

y_pred_grid_dtr1=grid_dtr1.predict(x_test)
grid_dtr1_score=grid_dtr1.score(x_test,y_test)
grid_dtr1_mae=mean_absolute_error(y_test,y_pred_grid_dtr1)

y_pred_rfc=rfc.predict(x_test)
rfc_score=rfc.score(x_test,y_test)
rfc_mae=mean_absolute_error(y_test,y_pred_rfc)


print(f' Linear Regression Score:{lr_score},Linear Regression mae : {linear_mae} \n\n KNN score : {KNN_score},KNN mae : {knn_mae} \n\n  Decition tree score : {dtr_score},Decition tree mae : {dtr_mae} \n\n  KNN grid score: {KNN_grid_score},KNN grid mae: {KNN_grid_mae} \n\n  Decition tree grid score : {grid_dtr_score},Decition tree grid mae : {grid_dtr_mae} \n\n  Decition tree grid1 score : {grid_dtr1_score},Decition tree grid1 mae : {grid_dtr1_mae} \n\n  Random forest score : {rfc_score},Random forest mae : {rfc_mae}')
 


**Model and Encoder Memory Files**

In [90]:
import pickle

In [91]:
with open("RandomForestRegressor.pkl",'wb') as RFC_file:
    pickle.dump(rfc,RFC_file)

In [70]:

with open("LabelEncoder_Area.pkl",'wb') as encoder_file:
    pickle.dump(lb_en_area,encoder_file)

In [71]:
with open("LabelEncoder_Item.pkl",'wb') as encoder_file:
    pickle.dump(lb_en_Item,encoder_file)